In [1]:
import os
import pandas as pd
import time
import pickle
import copy

In [2]:
import matgl
from matgl.ext.ase import PESCalculator, MolecularDynamics, Relaxer

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [6]:
import torch
torch.set_num_threads(4)

In [8]:
torch.get_num_threads()

4

In [9]:
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_std", torch.tensor(da

In [10]:
relaxer_matgl = Relaxer(pot)

In [11]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [12]:
def relaxer_2021_2_8_PES(structure):
    res = relaxer_matgl.relax(structure)
    structure = res['trajectory'].atoms.copy()
    structure.calculator = None
    return structure, res['trajectory'].energies[-1]

def relaxer_direct_2021_2_8_PES(row, log_file = None):
    structure_for_relax = copy.deepcopy(row['init_structure'])
    return relaxer_2021_2_8_PES(structure_for_relax)
    
def relaxer_jitter_c_005_2021_2_8_PES(row, log_file = None):
    structure_for_relax = copy.deepcopy(row['init_structure'])
    structure_for_relax = jitter_ic(structure_for_relax)
    structure_for_relax = jitter_ia(structure_for_relax)
    return relaxer_2021_2_8_PES(structure_for_relax)

## creating a dabase 'df_calc' and running the direct relaxation in M3GNet model

In [13]:
from disalloy.grid_run import run_packet

In [14]:
# df = structure_DB.copy()
if os.path.exists('df_calc.pkl'):
    df = pd.read_pickle('df_calc.pkl')

In [15]:
relaxer_dic_2021_2_8_PES = {'calculator_label': 'relaxer_direct_2021_2_8_PES',
                            'relaxer': relaxer_direct_2021_2_8_PES}

In [16]:
print(df.attrs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-08 08:35:43 - starting packet run: attempting 10 runs of size 1

2024-09-08 08:35:43 - no more structures to run of this size

0.35466694831848145s


In [18]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 20, 2)
print('{}s'.format(time.time() - ti))

2024-09-08 08:35:55 - starting packet run: attempting 20 runs of size 2

2024-09-08 08:35:55 - attempting to run :
 [33, 36, 37, 38, 40, 45, 15, 49, 18, 19, 53, 54, 58, 59, 29, 63]
of size :
 16
with calculator :
 relaxer_direct_2021_2_8_PES


2024-09-08 08:37:45 - Succesfuly ran packet of 16 runs in 53.37328290939331s

110.19736766815186s


In [20]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 20, 2)
print('{}s'.format(time.time() - ti))

2024-09-08 08:38:54 - starting packet run: attempting 20 runs of size 2

2024-09-08 08:38:54 - no more structures to run of this size

0.39673781394958496s


In [22]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 90, 3)
print('{}s'.format(time.time() - ti))

2024-09-08 08:40:35 - starting packet run: attempting 90 runs of size 3

2024-09-08 08:40:35 - attempting to run :
 [163, 339, 107, 188, 219, 291, 389, 251, 391, 318, 196, 289, 399, 263, 347, 324, 180, 285, 77, 121, 303, 114, 203, 161, 270, 111, 164, 377, 298, 384, 177, 100, 192, 96, 108, 193, 295, 396, 279, 198, 288, 372, 278, 252, 312, 254, 104, 375, 256, 232, 221, 330, 73, 242, 388, 249, 398, 223, 144, 191, 218, 307, 178, 309, 265, 365, 129, 306, 361, 103, 248, 217, 169, 113, 215, 185, 362, 226, 69, 72, 345, 167, 87, 74, 127, 231, 349, 175, 397, 267]
of size :
 90
with calculator :
 relaxer_direct_2021_2_8_PES


2024-09-08 08:46:53 - Succesfuly ran packet of 90 runs in 335.57671570777893s

378.47462582588196s


## now we run with another relaxer (using cell relaxer), but we calculate the same indeces previously calculated

In [17]:
relaxer_dic_jitter_c_005_2021_2_8_PES = {
    'calculator_label': 'relaxer_dic_jitter_c_005_2021_2_8_PES',
    'relaxer': relaxer_jitter_c_005_2021_2_8_PES
}

In [18]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 10, 1, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-04 01:59:03 - starting packet run: attempting 10 runs of size 1

2024-09-04 02:00:20 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-04 02:00:20 - Succesfuly ran packet of 8 runs in 45.17092323303223s

76.77451825141907s


In [21]:
calculated_index = df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()].index


In [22]:
for i, j, k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_dic_2021_2_8_PES'].array, 
             df.loc[calculated_index, 'relaxer_dic_jitter_c_005_2021_2_8_PES'].array):
    print(compute_distortion(i, j[0]))
    print(compute_distortion(i, k[0]))
    print()

1.5728799997364826e-06
0.008478383019435431

3.4711914702793634e-06
0.014992121608954932

1.869416926911698e-05
0.0055260174383238575

6.591054889187993e-06
0.0013269749304700654

2.8925424054246692e-05
0.041815786530901525

1.5884375891685432e-05
0.016760385825690095

2.5211789102510017e-06
0.002978565993329864

2.0687255793244462e-05
0.2675571670462636

0.16736472938249838
0.1556391981118748

0.0033938860022789045
0.005400236771320762

0.08171670991394009
0.08455231601816081

0.001232368835267994
0.006085425992822564

0.10549966456578831
0.10507149580550249

0.02025507197885123
0.022524359721694635

0.0801752265765604
0.08018755315184305

0.27751017164710384
0.2743531062992893

0.15023570149026255
0.14603844484541006

0.04627730708397882
0.04563094194938786

0.28299233544182945
0.2952608167347834

0.148931955963863
0.13934529862433764

0.2906526056232204
0.2871462242578758

0.15756575302427603
0.15675519381732916

0.16465976636159527
0.1634978212083279

0.029787477574621146
0.0323071